<a href="https://colab.research.google.com/github/arielmeragelman/PracticaProfesionalizante2/blob/testing/test_proyecto2022/testing_sobre_proyecto_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SyntaxError: invalid syntax (3750702446.py, line 1)

In [ ]:
# librerias requeridas:
# PyMySQL
# pandas
# BeautifulSoup

#DEFINO LOS DATOS PARA LA CONEXION CON LA BASE DE DATO DONDE SE GUARDA LA INFORMACION DEL SCRAPPING

import itertools
import pymysql 
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
import numpy as np
import seaborn as sb
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score




# Parametros de conexion a bbdd
link=pymysql.connect(
   host="192.141.210.12",    
   user="laboratorio", 
   password="laboratorio",
   db="bd_1", 
   charset="utf8",
   port=3306
   )

def strdate_to_floatdate(data):
#Funcion para convertir los indices de fecha en str a indices en float considerando la unidad el año y como decimales los meses (mes/12) y los dias (dias/30)
    return int(data[0:4])+int(data[5:7])/120+int(data[8:10])/3000
def regresion_lineal(x,y,j):
    X_train=np.array(list(map(strdate_to_floatdate,x)))
    fit = np.polyfit(X_train, y, deg=1)
    print ("Slope : " + str(fit[0]))
    print ("Intercept : " + str(fit[1]))
    
    
    #Fit function : y = mx + c [linear regression ]
    fit_function = np.poly1d(fit)

    #Linear regression plot
    plt.plot(x, fit_function(X_train))
    #Time series data plot
    plt.plot(x, Y_train)

    plt.xlabel('Fecha')
    plt.ylabel('Precio')
    plt.title('Cluster N°'+str(j))
    plt.show()
    return fit_function


def scrap(año, mes, url):
    # funcion para hacer un web scrapping con el valor del dolar
    
    for i in range(1,7):
        try:        
            fecha = datetime.datetime(año,mes,i)
            data = {'fecha': fecha.strftime('%d-%m-%y')}
            resp = requests.post(url, data=data)
            soup = BeautifulSoup(resp.text, "html.parser")
            break
        except:
            print('Falló en ',i)    
    filas = soup.find_all('td', {'style' : 'padding: 1%'})
    return filas

def parsear(filas):
  # Funcion para convertir el formato de los datos 
    mensual = pd.DataFrame() 
    for i in range(1, int(len(list(filas))/3)):
        dic = {}
        dic['fecha'] = filas[3*i].text
        dic['bid'] = filas[3*i+1].text
        dic['ask'] = filas[3*i+2].text
        rueda = pd.DataFrame.from_dict(dic, orient='index').transpose().set_index('fecha')
        rueda.index = pd.to_datetime(rueda.index, format='%d-%m-%y ')
        mensual = pd.concat([mensual,rueda], axis=0)
    return mensual

def downloadAño(año,url):
    tablaAnual = pd.DataFrame()
    for i in range(1,13):
        filas = scrap(año=año, mes=i,url=url)
        tabla = parsear(filas)
        tablaAnual = pd.concat([tablaAnual,tabla],axis=0)
        print('mes',i,'listo')        
    tablaAnual.to_excel('blue_'+str(año)+'.xlsx')
    print(tablaAnual)
    return tablaAnual




def analisis_kmean(matriz,k):
        # GENERAMOS CENTROIDES PARA 5 CLUSTERS
    kmeans = KMeans(n_clusters=k).fit(X)
    centroids = kmeans.cluster_centers_
    
    
    print("_"*20)
    print("-"*20)
    print("CENTROIDES PARA K = {}".format(k))
    print(centroids)
    
    
    #vemos el representante del grupo, el usuario cercano a su centroid
    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)
    print("_"*20)
    print("-"*20)
    print("REPRESENTANTES DE CADA GRUPO DE K = {}".format(k))
    print(closest)
    
    
    # Producto que representa a cada centroide
    print("_"*20)
    print("-"*20)
    
    n_centroides=[]
    for row in closest:
        print(Y[row])
        n_centroides.append(Y[row])
    PD_centroides= pd.DataFrame({'Productos':n_centroides})
    
    PD_centroides.to_csv("KMEAN_Centroides"+str(k)+"K_V1.csv")

        
    
    # Desarrollamos una lista de las categoria asociada a cada producto 
    categoria=[]
    for i in range(0,matriz.shape[0]):
        lugar=kmeans.predict( np.array( [ matriz.iloc[i,:] ]  ) )
        lugar=list(lugar)[0]
        categoria.append(lugar)
    
    PD_Categorias= pd.DataFrame({'Productos':matriz.index.values,'Categoria':categoria})
    
    PD_Categorias.to_csv("KMEAN_"+str(k)+"K_V1.csv")
    return PD_Categorias


#DEFINO LA CONSULTA PARA LEER LA INFORMACION DE LA BASE DE DATOS


def consulta_lineas(sql,link):
  
    cursor = link.cursor()
    cursor.execute(sql)
    print(cursor.rowcount, "LECTURA CORRECTA")
    salida=[]
    for row in cursor.fetchall():
        print(row)
        salida.append(row)
    cursor.close()
    return salida
    

#EJECUTO UNA CONSULTA MODELO PARA EL ANALISIS DE DATOS
sql = """select distinct N_Producto,Precio_Origen,FECHA from entradas_super where N_Super = "SUPER MAMI" order by N_Producto,Fecha ASC  """
#OBTENGO UNA LISTA CON LAS SALIDAS DE LA QUERY
tabla=consulta_lineas(sql,link)

# CONVIERTO LA LISTA EN UN DATAFRAME
a= pd.DataFrame(tabla)


# DEFINO DOS ARREGLOS UNIDIMENCIONALES CON LAS FECHAS Y LOS PRODUCTOS SIN QUE SE REPITAN
nuevas_columnas=a[0].unique()
fechas= a[2].unique()


# CREO UNA MATRIZ QUE OBTIENE LOS VALORES DE LA "tabla"  PERO LOS ORDENA PONIENDO CADA ELEMENTO DE FORMA UNICA E IRREPETIBLE SEGUN EL NOMBRE DEL PRODUCTO Y LA FECHA
# QUEDANDO LA FECHA COMO INDICE DE FILAS Y LOS PRODUCTOS COMO INDICE DE COLUMNAS

matriz_nueva=[]
for j,dia in enumerate(fechas):
    matriz_nueva.append([])
    for i,columna in enumerate(nuevas_columnas):
            try:
                valor=   float(a[ (a[0]==columna) & (a[2]==dia) ][1] )
            except TypeError:
                    valor=   0
            
           # print("En la posicion i:"+str(i)+", j: "+str(j)+" Tenemos el valor: "+str(valor))
            matriz_nueva[j].append(valor)
            
print("FINALIZADO")

#CONVIERTO A LA MATRIZ EN UN DATAFRAME
matriz_nueva=pd.DataFrame(matriz_nueva)
#DEFINO EL NOMBRE DE LAS COLUMNAS
matriz_nueva.columns=nuevas_columnas
#DEFINO LOS INDICES DE FILAS
matriz_nueva.index=fechas
#DEFINO EL NOMBRE DEL INDICE DE FILA
matriz_nueva.index.name="fechas"
matriz_nueva.to_csv("resguardo_csv")



# OBTENGO 2 DATAFRAME CON LOS VALORES HISTORICOS DEL DOLAR EN ARGENTINA

año=2022

dolar_blue= downloadAño(año,'https://www.cotizacion-dolar.com.ar/dolar-blue-historico-'+str(año)+'.php')
dolar_oficial= downloadAño(año,'https://www.cotizacion-dolar.com.ar/dolar-historico-bna-'+str(año)+'.php')

dolar=pd.merge(dolar_blue, dolar_oficial, on='fecha')

# Cambio el formato del index de la matriz obtenida por web scrapping para que el indice sea en formato datetime y poder unirlo con la matriz del precio del dolar
matriz_nueva.index = pd.to_datetime(matriz_nueva.index)

matriz_con_dolar=pd.merge(matriz_nueva, dolar, left_index=True, right_index=True, how='left')


# CONVIERTO LOS OBJETOS EN VALORES FLOTANTES PARA PODER COMPARAR LA VARIACION

for i in matriz_con_dolar.columns:
       if matriz_con_dolar[i].dtype == 'O':
            try:
                matriz_con_dolar[i]=matriz_con_dolar[i].str.strip('$ ').astype(float)
            except:
                passmatriz_con_dolar


# COMENZAMOS CON LA LIMPIEZA DE VALORES IRRACIONALES
# CONVERTIMOS TODOS LOS PRECIOS INFERIORES A $2 EN NAN

import numpy as np

for i in range(len(nuevas_columnas)):
   
    indice=nuevas_columnas[i]
    matriz_con_dolar.loc[  matriz_con_dolar[indice] < 10, indice    ] = np.nan




# COMPLETAMOS CON DATOS LOS CAMPOS VACIOS
matriz_con_dolar=matriz_con_dolar.fillna(method="ffill")
matriz_con_dolar=matriz_con_dolar.fillna(method="backfill")



#Doble chequeamos los minimos para quitar posibles errores de scrapping
for columna in matriz_con_dolar.columns:
    
    print("minimo de {}: {}".format(columna,matriz_con_dolar[columna].min()))


# ELIMINAMOS LAS COLUMNAS QUE ESTEN COMPLETAMENTE INUTILIZABLES
nuevas_columnas=matriz_con_dolar.columns

for v,i in enumerate(nuevas_columnas):
    if matriz_con_dolar[i].isna().all() :
        del matriz_con_dolar[i]


# DEFINIMOS EL NUEVO ARREGLO DE COLUMNAS  CONSIDERANDO QUE HEMOS QUITADO ALGUNAS DE SU ARREGLO ORIGINAL

nuevas_columnas=matriz_con_dolar.columns

matriz_con_dolar=matriz_con_dolar.sort_index()

# Creamos una copia para poder realizar pruebas sin dañar la informacion original
matriz_con_dolar_bk=matriz_con_dolar.copy()



# <<<<<----- OJO HAY QUE VERIFICAR QUE EL CONTROL DE DUPLICADO LO HAGA SI SE REPITEN LOS VALORES EN TODDA LA FILA

# VAMOS A VERIFICAR SI HAY COLUMNAS QUE TENGAN LOS MISMOS PRECIOS EN TODAS LAS FILAS,
#DE SER ASI SE PODRIA  TRATAR DE UN MISMO PRODUCTO CON DISTINTAS DENOMINACIONES Y QUE
#NO APORTARA INFORMACION UTIL

duplicados =[]

for col in range(matriz_con_dolar.shape[1]):
        contents= matriz_con_dolar.iloc[:, col]
        
        for comp in range( col + 1,matriz_con_dolar.shape[1]):
            if contents.equals(matriz_con_dolar.iloc[:, comp]):
                duplicados.append(comp)
                
duplicados = np.unique(duplicados).tolist()
      
matriz_con_dolar=matriz_con_dolar.drop( matriz_con_dolar.iloc[:, duplicados].columns,axis=1)

#PUNTO DE RESGUARDO DE LOS DATOS
matriz_con_dolar.to_csv("Matriz_precios_back.csv")


matriz_con_dolar=matriz_con_dolar.drop('indicator',axis=1)


#  Genero una matriz de valores con la variacion de precios entre registros de una misma categoria
matriz_variacion=(matriz_con_dolar/matriz_con_dolar.shift(1))-1

# Eliminamos la primer fila ya que no tiene valores comparativos de utilidad
matriz_variacion=matriz_variacion.iloc[1: , :]



no_varian=[]
for i in matriz_variacion:
    if len(matriz_variacion[i].unique()) == 1:
        print("SIN VARIACION DE PRECIOS "+i)
        no_varian.append(i)
#print("NO varian: "+str(varian))
print("total productos: "+str(len(matriz_variacion.columns)))


#Eliminamos las columnas que no presentan variacion de precio

print(matriz_variacion.shape)

for i in no_varian:
    try:
        matriz_variacion = matriz_variacion.drop(columns=[i], axis = 1)
    except:
        pass

print("Ya se eliminaron las columnas")
print(matriz_variacion.shape)

matriz_variacion_acumulada= matriz_variacion.cumsum()

# RESGUARDAMOS LOS DATOS
matriz_variacion_v1 = matriz_variacion_acumulada.copy()
# Eliminamos el indice de fechas para trabajar el modelo de datos como mediciones individuales y no como una serie de tiempo
matriz_variacion_v1=matriz_variacion_v1.reset_index(drop=True)

matriz_variacion_v1.head()

matriz_variacion_v1.to_csv("matriz_variacion_v1.csv")

# Inicio clusterizacion de datos
T_matriz_variacion_v1 = matriz_variacion_v1.T
T_matriz_variacion_v1.to_csv("backup_matriz_variacion")

#Armamos el conjunto de datos para trabajar en el proceso de clusterizado
Y=matriz_variacion_v1.index.values
X=np.array(matriz_variacion_v1)

kmeans = KMeans(n_clusters=6).fit(X)
centroids = kmeans.cluster_centers_

closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, X)
# Creamos una lista "categoria" donde predecimos la categoria de todos los productos que tenemos disponibles
categoria=[]
for i in range(0,480):
    lugar=kmeans.predict( np.array( [ matriz_variacion_v1.iloc[i,:] ]  ) )
    lugar=list(lugar)
    categoria.append(lugar)


# Unimos en una sola lista a todos los productos

categoria=list(itertools.chain(*categoria))
print(categoria)

# Creamos un Dataframe de dos columnas, producto y su categoria
PD_Categorias_K6= pd.DataFrame({'Productos':matriz_variacion_v1.index.values,'Categoria':categoria})

# Respaldamos en csv
PD_Categorias.to_csv("KMEAN_6K_V1.csv")

analisis_kmean(matriz_variacion_v1,6)

#cargamos los datos de entrada
indice = pd.read_csv("KMEAN_Centroides6K_V1.csv",index_col=[0])
data = pd.read_csv("Matriz_precios_back.csv",index_col=[0])

centroides_k6=[]
for i in range(indice.shape[0]): 
    centroides_k6.append(data[indice.iloc[i,:][0]])

fit_f=[]
for i in range(len(centroides_k6)):
    X_train=np.array(centroides_k6[i].index)
    Y_train=np.array(centroides_k6[i].values)
    fit_f.append(regresion_lineal(X_train,Y_train,i))


for i in range(len(centroides_k6)):
    X_train=np.array(centroides_k6[i].index)

    fit_function=fit_f[0]
    prediction = fit_function(X_train.size + 14)
    print(prediction)